In [1]:
import os
import glob
import torch
import SimpleITK as sitk

import numpy as np
from scipy.ndimage import gaussian_filter, binary_erosion

c:\Users\user\anaconda3\envs\sjs\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
path = f"D:\\HIPPO\\DATA\\"

ct, hippo = [], []

for folder in os.listdir(path):
    folder_path = f"{path}{folder}\\"
    for file in glob.glob(folder_path + f"/*.nii"):
        filename = file.split('\\')[4]
        if filename.startswith('ori'):
            ct.append(file)
        elif filename.startswith('lh+rh'):
            hippo.append(file)

In [ ]:
def get_volume(path):
    volume = sitk.ReadImage(path)
    volume = sitk.GetArrayFromImage(volume)

    volume = np.transpose(volume, (1, 0, 2))
    volume = np.rot90(volume, 2)

    return volume

def minmaxnormalize(volume):
    copy_volume = volume.copy()

    s = np.min(volume)
    b = np.max(volume)

    return (copy_volume - s) / (b - s)

def adjust__window(volume):
    copy_volume = volume.copy()

    window_min = -20
    window_max = 100

    copy_volume[copy_volume <= window_min] = window_min
    copy_volume[copy_volume >= window_max] = window_max

    return copy_volume

def get_binary_volume(volume, threshold=None):
    copy_volume = volume.copy()

    if threshold is None:
        copy_volume[copy_volume != 0] = 1
    else:
        copy_volume[copy_volume <= threshold] = 0
        copy_volume[copy_volume >= threshold] = 1

    return copy_volume

def get_boundary_map(volume):
    filter_data = gaussian_filter(volume, 1)
    threshold = 0.4

    binary_mask = filter_data > threshold

    eroded_mask = binary_erosion(binary_mask)
    boundary_map = binary_mask.astype(int) - eroded_mask.astype(int)

    return boundary_map

def crop__volume(volume, crop_size):
    copy_volume = volume.copy()

    d, h, w = volume.shape
    
    start_z = d // 2
    start_x = h // 2
    start_y = w // 2

    cropped_volume = copy_volume[start_z - crop_size[0] // 2 : start_z + crop_size[0] // 2,
                                start_x - crop_size[1] // 2 : start_x + crop_size[1] // 2,
                                start_y - crop_size[2] // 2 : start_y + crop_size[2] // 2,]
    
    return cropped_volume

In [ ]:
def data_preprocessing(ctpath, hippopath):

    ct = get_volume(ctpath)
    hippo = get_volume(hippopath)

    ct = crop__volume(ct, (96,128,128))
    hippo = crop__volume(hippo, (96,128,128))

    ct = adjust__window(ct)
    ct = minmaxnormalize(ct)

    hippo = get_binary_volume(hippo)
    boundary = get_boundary_map(hippo)

    boundary = crop__volume(boundary, (96,128,128))

    return ct, hippo, boundary